<a href="https://colab.research.google.com/github/narsym/Advanced-Python/blob/master/Fluent_python_chap_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Functions as first class objects

In [1]:
def factorial(n):
  '''returns n'''
  return 1 if n < 2 else n * factorial(n - 1)

In [2]:
factorial(10)

3628800

In [3]:
factorial.__doc__

'returns n'

In [4]:
type(factorial)

function

In [5]:
help(factorial)

Help on function factorial in module __main__:

factorial(n)
    returns n



Assign function to a particular variable

In [6]:
fact = factorial
print(fact(10))
list(map(fact, range(10)))

3628800


[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

Higher order functions (take function as an argument and return function)

In [7]:
fruits = 'strawberry apple fig banana orange mango'.split()
sorted(fruits, key = len) # sorted higher order function

['fig', 'apple', 'mango', 'banana', 'orange', 'strawberry']

In [8]:
def reverse(word):
  return word[::-1]

sorted(fruits, key = reverse)

['banana', 'orange', 'apple', 'fig', 'mango', 'strawberry']

some higher order functions are map, filter, reduce

__Modern replacements for these higher order functions are:__

_list comprehensions and generator expressions_

In [11]:
print(list(map(fact, range(10))))#map
print([fact(n) for n in range(10)])#listcomp

print(list(map(fact, filter(lambda n: n % 2, range(10)))))#map, filter and lambda
print([fact(n) for n in range(10) if n % 2])#listcomp,  same thing can done independently in genexps

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]
[1, 6, 120, 5040, 362880]
[1, 6, 120, 5040, 362880]


Anonymous functions

In [13]:
sorted(fruits, key = lambda word: word[::-1]) #lambda reverse a string and return it

['banana', 'orange', 'apple', 'fig', 'mango', 'strawberry']

UserDefined callable functions

In [18]:
import random
class BingoCage:
  def __init__(self, items):
    self._items = list(items)
    random.shuffle(self._items)

  def pick(self):
    try:
      return self._items.pop()
    except IndexError:
      raise LookupError('pick from empty bingo cage')

  def __call__(self):
    return self.pick()

In [19]:
bingo = BingoCage(range(10))
print(bingo.pick()) #calling pick method
print(bingo())    #shortcut to bingo.pick() method

8
5


In [21]:
def tag(name, *content, cls = None, **attrs):
  """Generate one or more HTML tags"""
  if cls is not None:
    attrs['class'] = cls
  if attrs:
    attr_str = ''.join(' %s="%s"' % (attr, value) for attr, value in sorted(attrs.items()))
  else:
      attr_str = ''
  if content:
    return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)
  else:
    return'<%s%s />' % (name, attr_str)

In [22]:
tag('br')

'<br />'

In [23]:
tag('p', 'hello')

'<p>hello</p>'

In [24]:
print(tag('p', 'hello', 'world', cls = 'sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [25]:
tag(content = 'testing', name = 'img')

'<img content="testing" />'

In [27]:
my_tag={'name':'img','title':'Sunset Boulevard','src':'sunset.jpg','cls':'framed'}
tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

Packages for functional programming

the operator module

In [30]:
from functools import reduce
from operator import mul

def fact(n): # using lambda
  return reduce(lambda a, b: a * b, range(1, 10))
fact(10)

362880

In [31]:
def fact(n): #using operator's in built methods
  return reduce(mul, range(1, 10))
fact(10)

362880

**itemgetter** gets the element present at that position from any class that implements 'getitem' special method

In [32]:

metro_data = [
             ('Tokyo','JP',36.933,(35.689722,139.691667)),
             ('Delhi NCR','IN',21.935,(28.613889,77.208889)),
             ('Mexico City','MX',20.142,(19.433333,-99.133333)),
             ('New York-Newark','US',20.104,(40.808611,-74.020386)),
             ('Sao Paulo','BR',19.649,(-23.547778,-46.635833))
]

In [33]:
from operator import itemgetter
for city in sorted(metro_data, key = itemgetter(1)):
  print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


If you pass multiple index arguments to itemgetter, the function it builds will return tuples with the extracted values:

In [37]:
cc_name = itemgetter(1, 0)
for city in metro_data:
  print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


In [38]:
from collections import namedtuple

LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')

metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long)) for name, cc, pop, (lat, long) in metro_data]
print(metro_areas[0].coord.lat)

35.689722


In [39]:
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [40]:
from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')
for city in sorted(metro_areas, key = attrgetter('coord.lat')):
  print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


functions in operator

In [43]:
import operator
[name for name in dir(operator) if not name.startswith('_')]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

methodcaller

In [44]:
from operator import methodcaller
s = 'the time has come'
upcase = methodcaller('upper')
print(upcase(s))
hibernate = methodcaller('replace', ' ', '-') #freeze some arguments, here in replace ' ' and '-'
print(hibernate(s))

THE TIME HAS COME
the-time-has-come


Freezing Arguments with functools.partial

In [45]:
from functools import partial
triple = partial(mul, 3)
triple(7)

21

In [46]:
list(map(triple, range(1, 10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]

In [57]:
import unicodedata, functools
nfc = functools.partial(unicodedata.normalize, 'NFC')
s1 = 'café'
s2 = 'cafe\u0301'
print(s1, s2)
print(s1 == s2)
print(nfc(s1) == nfc(s2))

café café
False
True
